In [47]:
pip install -U tensorflow-text==2.15.*

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.15.* (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.15.*


In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow_text'

In [33]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


## Data Preprocessig 

In [9]:
df = pd.read_csv("spam.csv")

In [10]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [12]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [13]:
747/4825

0.15481865284974095

###### Only 15% of the Data Contains Spam Emails and 85% of the Data Contains Ham Email.

##### The DataSet is Clearly Imbalanced

In [14]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

###### No Missing Values

In [15]:
df_spam = df[df['Category']=='spam']

In [16]:
df_ham = df[df['Category']=='ham']

In [17]:
print(df_spam.shape)

(747, 2)


In [18]:
print(df_ham.shape)

(4825, 2)


In [19]:
ham_down_sampled = df_ham.sample(df_spam.shape[0])
ham_down_sampled.shape

(747, 2)

In [20]:
df_balanced = pd.concat([df_spam,ham_down_sampled])
df_balanced['Category'].value_counts()

Category
spam    747
ham     747
Name: count, dtype: int64

In [21]:
df_balanced.shape

(1494, 2)

##### Although, it ain't a Good Practice to Remove all the Valuable Training Data by Down Sampling the Ham
##### But Here It Works Well With BERT

In [22]:
df_balanced.sample(5)

,Category,Message
450,ham,10 min later k...
67,spam,Urgent UR awarded a complimentary trip to Euro...
4001,spam,This is the 2nd time we have tried to contact ...
5035,ham,I want to go to perumbavoor
1847,ham,Its on in engalnd! But telly has decided it wo...


In [23]:
df_balanced['Encoded'] = df_balanced['Category'].apply(lambda x: 1 if x =='spam' else 0)

In [24]:
df_balanced.sample(10)

,Category,Message,Encoded
1327,ham,"Sorry, was in the bathroom, sup",0
5028,spam,PRIVATE! Your 2003 Account Statement for shows...,1
876,spam,"Shop till u Drop, IS IT YOU, either 10K, 5K, £...",1
297,ham,Unless it's a situation where YOU GO GURL woul...,0
3456,ham,No need lar. Jus testing e phone card. Dunno n...,0
3621,ham,I meant as an apology from me for texting you ...,0
2408,ham,Solve d Case : A Man Was Found Murdered On &l...,0
4500,spam,"SMS SERVICES. for your inclusive text credits,...",1
3598,spam,Congratulations YOU'VE Won. You're a Winner in...,1
5489,ham,Oh thanks a lot..i already bought 2 eggs ..,0


In [25]:
X = df_balanced['Message']
y = df_balanced['Encoded']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,stratify=y)

In [27]:
X_train.head(5)

2034                              IM GONNA MISS U SO MUCH
652     I'm gonna say no. Sorry. I would but as normal...
314     Hi the way I was with u 2day, is the normal wa...
3152                        Yeah but which is worse for i
240     U 447801259231 have a secret admirer who is lo...
Name: Message, dtype: object

In [28]:
bem = hub.load('https://kaggle.com/models/google/bert/frameworks/TensorFlow2/variations/answer-equivalence-bem/versions/1')

In [29]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

RuntimeError: Op type not registered 'CaseFoldUTF8' in binary running on LAPTOP-5VT13OTP. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.